# Example 7: Measuring worm length

Body length is a commonly measured trait. However, live animals are typically not well behaved in front of camera: typically the body is not aligned straight, which makes length measure difficult and sometimes time consuming. Some computer vision can help here, such as skeletonization (https://en.wikipedia.org/wiki/Topological_skeleton) for an acurate semi automatic approach, and polyline drawing for a less acurate, manual aproach.

Here I demonstrate how to automatically measure the length of california blackworms (*Lumbriculus variegatus*) in 6 well plates by dragging a mask around them. A future implementation will like include a variant of [Hough transformation](https://en.wikipedia.org/wiki/Hough_transform) to automatically detect the circles of the wellplates for a full automatic workflow. 

**Note:** For details on the differen workflows (prototyping-, low throughput- and high throughout) see [Tutorial 2](tutorial_2_phenopype_workflow.ipynb). Not all examples include the prototyping workflow.

<div class="row; text-align: left">
    
<div class="col-md-6">
    
![Before](_assets/ex7_before.jpg)
    
**Input** - Worms placed in 6 well plates
</div>
<div class="col-md-6">

![After](_assets/ex7_after.jpg)
    
**Results** - extracted body contour (green area) and body length (red line) as a topological skeleton
</div>
</div>


## Low throughput

First we load the image as a container. If you have saved data before with a specific `save_suffix` (recommended for reproducibility and to not overwrite files) you can reload it if you have provided it at the point of container creation in `load_image`:

In [11]:
import phenopype as pp

image_path = r"./images/worms.jpg"
custom_save_path = r"../_temp/output"
ct = pp.load_image(image_path, # path to image
                   cont=True,  # create a phenopype container
                   save_suffix="worms1",      # indicate a suffix for save- and loading operations
                   dirpath=custom_save_path)  # supply an (existing) directory where files get stored and loaded from

In [12]:
ct.load() # to reload any saved masks or polylines

AUTOLOAD
- drawing loaded from attributes.yaml
- polylines_worms1.csv
- masks_worms1.csv


In [13]:
ct.reset() # reset canvas and images, if you run repeatedly
pp.preprocessing.create_mask(ct) # finding circles can be potentially automatized in the future using hough transform
pp.segmentation.threshold(ct, method="adaptive", blocksize=49, constant=5)
pp.segmentation.find_contours(ct, min_diameter=50)
## pp.segmentation(draw) # uncomment this if you need to manually separate worms from the well-border
## pp.visualization.polylines(ct) # uncomment this if you need to manually measure a worm
pp.measurement.skeletonize(ct) # this will measure the longest distance in a curvy object
pp.visualization.draw_contours(ct) # this draws both the contour (green) and the skeleton
pp.visualization.draw_masks(ct) # will be skipped if loaded before
pp.export.save_masks(ct, dirpath=custom_save_path) # 
pp.export.save_contours(ct, dirpath=custom_save_path)
pp.export.save_canvas(ct, dirpath=custom_save_path, resize=1) # resize=1 to have the original sized picture

- mask with label mask1 already created (overwrite=False)
- include mask "mask1" pixels
- include mask "mask1" pixels
- include mask "mask1" pixels
- include mask "mask1" pixels
- include mask "mask1" pixels
- include mask "mask1" pixels
 - show mask: mask1.
 - show mask: mask1.
 - show mask: mask1.
 - show mask: mask1.
 - show mask: mask1.
 - show mask: mask1.
- masks saved under ../_temp/output\masks_worms1.csv (overwritten).
- contours saved under ../_temp/output\contours_worms1.csv (overwritten).
- canvas saved under ../_temp/output\canvas_worms1.jpg (overwritten).


In [14]:
pp.show_image(ct.canvas)

Should a worm superimpose itself it is not possible for the skeletonization algorithm to follow the shape through from beginning to end (e.g. in worm 3 in this example). In that case one has to manually draw a line along the worm using the `polylines` tool. 

In [16]:
pp.measurement.polylines(ct)
pp.visualization.draw_polylines(ct)
pp.export.save_polylines(ct)

- polylines already drawn (overwrite=False)
- polylines saved under ../_temp/output\polylines_worms1.csv (overwritten).


In [17]:
pp.show_image(ct.canvas)

Alternatively to the `polylines` tool, one can also use the drawing tool if you just need to separte an endpoint or if the worm touches the border. This should be done on the binary image, *before* the contour are detected. 

## High throughput

As for the other examples I have created a preset (`ex7`) with appropriate settings for the example. The template can be passed to the `pype` using `config_preset="ex7"` - see below. 

In [18]:
import phenopype as pp

image_path = r"./images/worms.jpg"

print(pp.presets.ex7)


preprocessing:
- create_mask
segmentation:
- threshold:
    method: adaptive
    blocksize: 49      ## for adaptive 
    constant: 5        ## for adaptive 
- find_contours:
    retrieval: ccomp
    min_diameter: 50
    min_area: 0
measurement:
- skeletonize
visualization:
- select_canvas:
    canvas: image
- draw_contours:
    line_width: 2
    label_width: 1
    label_size: 1
    fill: 0.3
export:
- save_contours:
    overwrite: true



In [23]:
pp.pype(image_path, name="worms2", dirpath=r"../_temp/output", config_preset="ex7")

../_temp/output\pype_config_worms2.yaml


------------+++ new pype iteration 2020:04:19 15:45:35 +++--------------


AUTOLOAD
- drawing loaded from attributes.yaml
- masks_worms2.csv
PREPROCESSING
create_mask
- mask with label mask1 already created (overwrite=False)
SEGMENTATION
threshold
- include mask "mask1" pixels
- include mask "mask1" pixels
- include mask "mask1" pixels
- include mask "mask1" pixels
- include mask "mask1" pixels
- include mask "mask1" pixels
draw
- polylines already drawn (overwrite=False)
find_contours
MEASUREMENT
skeletonize
VISUALIZATION
select_canvas
- invalid selection - defaulting to raw image
draw_contours
EXPORT
save_contours
- contours saved under ../_temp/output\contours_worms2.csv (overwritten).
save_canvas
- canvas saved under ../_temp/output\canvas_worms2.jpg (overwritten).
AUTOSAVE
save_masks
- masks not saved - file already exists (overwrite=False).
save_drawing
- drawing saved (overwriting)


TERMINATE
